<a href="https://colab.research.google.com/github/RyanChen12035/w266-NLP/blob/main/w266_final_project_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Package installation and importation

In [1]:
!pip install pydot --quiet
!pip install tensorflow-datasets --quiet
!pip install transformers --quiet

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-1980ebfa-5837-2b71-c621-3f19cb8bbe40)


In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re

Download IMDB review data and split them into training and testing(validation)

In [5]:
train_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:80%]', 'test[80%:]'),
    as_supervised=True)

train_examples, train_labels = next(iter(train_data.batch(20000)))
test_examples, test_labels = next(iter(test_data.batch(5000)))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYWZEOI/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYWZEOI/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYWZEOI/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_examples[:4]

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [ ]:
train_labels[:4]

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 0, 0, 1])>

In [6]:
MAX_SEQUENCE_LENGTH = 100

In [30]:
#allow us to get the hidden layer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased', output_hidden_states=True)

Training/Testing data tokenization

In [8]:
#BERT Tokenization of training and test data
#Embedding size of Bert tokenizer: 768
#Dictionary size of Bert tokenizer: 28,996


train_examples_str = [x.decode('utf-8') for x in train_examples.numpy()]
test_examples_str = [x.decode('utf-8') for x in test_examples.numpy()]

bert_train_tokenized = bert_tokenizer(train_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_train_inputs = [bert_train_tokenized.input_ids,
                     bert_train_tokenized.token_type_ids,
                     bert_train_tokenized.attention_mask]
bert_train_labels = np.array(train_labels)

bert_test_tokenized = bert_tokenizer(test_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_test_inputs = [bert_test_tokenized.input_ids,
                     bert_test_tokenized.token_type_ids,
                     bert_test_tokenized.attention_mask]
bert_test_labels = np.array(test_labels)

In [ ]:
train_examples_str[:2]

["This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
 'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development wa

In [31]:
#12 layers of transformer
#A drop out layer + dense layer with 100 hidden layer size on top + final layer with sigmoid as activation function

def create_bert_cls_model(bert_base_model,
                          max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 100,
                          dropout=0.3,
                          learning_rate=0.00005,
                          output_cls_tokens=False):
    """
    Build a simple classification model with BERT. Use the CLS Token output for classification purposes.
    """

    ### YOUR CODE HERE

    bert_base_model.trainable = True

    #input layers of BERT, shape (batch, max_sequence_length), model will be fit with bert_train_tokenized
    input_ids = Input(shape=(max_sequence_length,), dtype=tf.int32, name='input_ids')
    token_type_ids = Input(shape=(max_sequence_length,), dtype=tf.int32, name='token_type_ids')
    attention_mask = Input(shape=(max_sequence_length,), dtype=tf.int32, name='attention_mask')

    inputs = [input_ids, token_type_ids, attention_mask]

    #BERT output, last_hidden_state shape (batch, max_sequence_length, embedding dimensions)
    bert_output = bert_base_model(input_ids=input_ids,
                                  token_type_ids=token_type_ids,
                                  attention_mask=attention_mask,
                                  output_hidden_states=output_cls_tokens)

    #Extract the CLS token's output, the embedding representation of first token of every sentence, shape(batch, embedding dimensions)
    cls_token_output = bert_output[0][:, 0, :] # CLS token output from the last layer

    #Add a dropout layer
    x = Dropout(dropout)(cls_token_output)

    #Add a fully connected layer for classification
    x = Dense(hidden_size, activation='relu')(x)

    #Final output layer for classification, assuming it's binary task
    output = Dense(1, activation='sigmoid')(x)


    # CLS output for each layer of transformer
    if output_cls_tokens:
        cls_outputs = [state[:, 0, :] for state in bert_output[2]] # CLS token outputs from all layers
        model_outputs = [output] + cls_outputs

    else:
        model_outputs = output


    #Model complie
    classification_model = Model(inputs=inputs, outputs=model_outputs)
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss='binary_crossentropy',
                                 metrics=['accuracy'])

    ### END YOUR CODE
    return classification_model

In [32]:
#bert_model
bert_cls_model_classification = create_bert_cls_model(bert_model, output_cls_tokens=False)
history_cls_bert= bert_cls_model_classification.fit(bert_train_inputs,
                                                    bert_train_labels,
                                                    epochs=2,
                                                    batch_size=8,
                                                    validation_data=(bert_test_inputs, bert_test_labels))

Epoch 1/2


2500/2500 [==============================] - 594s 216ms/step - loss: 0.4267 - accuracy: 0.7997 - val_loss: 0.3584 - val_accuracy: 0.8312
Epoch 2/2
2500/2500 [==============================] - 517s 207ms/step - loss: 0.2773 - accuracy: 0.8855 - val_loss: 0.3862 - val_accuracy: 0.8446


In [ ]:
tf.keras.backend.clear_session()

# Model parsing
###### 1. Refering to IMDB reviews, split the task of predicting sentiment into five sub-tasks and generate the corresponding test reviews.
###### 2. Applying logit lens method to find out the which layer of transformer is triggered by the test reviews.
###### 3. Modifiy the model so it can generate CLS outputs of each transformer layer inside the fine-tuned BERT model.
###### 4. Compare CLS from each layer with final CLS output and find out when the model start to understand the test review is about positive or negative. In other word, when the model understand the sub task?
###### 5. Dive into the neurons in hidden layer of FFN, and find out which neuron plays a major role of understanding the sub task.


In [26]:
# Example test reviews
"""
1. Identifying Emotional Tone
Sub-Task: Determine the emotional tone of the review (e.g., positive, negative, neutral).
Test Reviews:
    "The movie's breathtaking scenery and exceptional soundtrack added depth to its rich storytelling." -> Positive Tone
    "The film was a letdown with its lackluster plot and uninspired performances." -> Negative Tone
2. Analyzing Subjective Statements
Sub-Task: Detect subjective statements or opinions in the review.
Test Reviews:
    "In my opinion, the film's portrayal of historical events was highly inaccurate." -> Subjective
    "The movie won three Academy Awards, including Best Picture." -> Objective
3. Evaluating Specific Aspects (Acting, Plot, Cinematography)
Sub-Task: Assess specific aspects of the movie like acting quality, plot development, and cinematography.
Test Reviews:
    "The acting was superb, with each character bringing depth and emotion to the screen." -> Positive Acting
    "The plot was predictable and lacked originality, making the movie quite boring." -> Negative Plot
4. Recognizing Extremes in Sentiment
Sub-Task: Identify reviews with extremely positive or negative sentiments.
Test Reviews:
    "This is possibly the worst movie ever made, with no redeeming qualities whatsoever." -> Extremely Negative
    "An absolute masterpiece, every moment was captivating and a joy to watch." -> Extremely Positive
5. Detecting Sarcasm or Irony
Sub-Task: Detect sarcasm or irony, which can often invert the apparent sentiment of a statement.
Test Reviews:
    "Oh great, another predictable rom-com, just what the world needs." -> Sarcasm
    "I loved how the movie ended abruptly without resolving any plot points." -> Irony
"""

# First reivews is positive tone and the second is negative tone
test_reviews = [
    "The movie's breathtaking scenery and exceptional soundtrack added depth to its rich storytelling.",
    "The film was a letdown with its lackluster plot and uninspired performances."
]

# Tokenize the reviews
# 101:[CLS], 102:[SEP]
token_inputs = bert_tokenizer(test_reviews,
                            max_length=MAX_SEQUENCE_LENGTH,
                            truncation=True,
                            padding='max_length',
                            return_tensors='tf')

inputs = [token_inputs.input_ids,
        token_inputs.token_type_ids,
        token_inputs.attention_mask]

print(inputs)

[<tf.Tensor: shape=(2, 100), dtype=int32, numpy=
array([[  101,  1109,  2523,   112,   188,  2184, 17181, 19335,  1105,
        12688,  5945,  1896,  5415,  1106,  1157,  3987, 25514,   119,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0],
       [  101,  1109,  1273,  1108,   170,  1519,  5455,  1114,  1157,
         2960, 23225,  4928,  1105,  8362,  4935, 23709,  3853,   119,
          10

In [27]:
#inference
outputs = bert_cls_model_classification(inputs)
outputs

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[9.9601424e-01],
       [3.6029250e-04]], dtype=float32)>

In [34]:
# Create a model for analysis which includes the hidden states
#bert_model is fine-tuned now, don't have to re-train it. it's an object.
cls_layer_inside_finetunedBERT = create_bert_cls_model(bert_model, output_cls_tokens=True)

#get cls in each layers of transformer inside the fine-tuned BERT
predictions, *cls_hidden_states = cls_layer_inside_finetunedBERT.predict(inputs)
print(cls_hidden_states)

1/1 [==============================] - 6s 6s/step
[array([[ 0.41741544,  0.11755207, -0.2406264 , ...,  0.06478093,
         0.00513756, -0.06817159],
       [ 0.41741544,  0.11755207, -0.2406264 , ...,  0.06478093,
         0.00513756, -0.06817159]], dtype=float32), array([[ 0.20473635, -0.05526565, -0.16731615, ...,  0.01225742,
        -0.01846263,  0.03241834],
       [ 0.22269574, -0.02642442, -0.14752658, ...,  0.03253958,
        -0.02308984,  0.02362826]], dtype=float32), array([[ 0.34505016,  0.02480805, -0.16118094, ..., -0.24086316,
        -0.00683834, -0.10608192],
       [ 0.3583671 , -0.00557777, -0.09305716, ..., -0.17521164,
         0.03325133, -0.19979668]], dtype=float32), array([[ 0.53194577,  0.24912241, -0.2695578 , ..., -0.10862309,
         0.12087879, -0.01178998],
       [ 0.59172565,  0.15989393, -0.03371595, ..., -0.4203649 ,
        -0.02867934,  0.1768316 ]], dtype=float32), array([[ 0.74141324,  0.5755651 , -0.6042033 , ...,  0.10887075,
         0.09911

In [43]:
#12 layers and final CLS
print(len(cls_hidden_states))

#The output of first layer of transformer. Shape (batch, embedding dimensions)
print(cls_hidden_states[0][:].shape)

13
(2, 768)


In [ ]:
#Next, find out how to compare the final one with each output